In [ ]:
#getting dictionaries of classifier.1 - pretrained Alexnet
!wget -nc https://www.dropbox.com/s/d3h6veqbxfcud5u/EarlyV_pretrained_alexnet_classifier1_feature_idxs.pickle?dl=0
!wget -nc https://www.dropbox.com/s/szfm8068hmyy1lx/pOTC_pretrained_alexnet_classifier1_feature_idxs.pickle?dl=0
!wget -nc https://www.dropbox.com/s/2ohi5ngsn6w8kch/aOTC_pretrained_alexnet_classifier1_feature_idxs.pickle?dl=0

#getting dictionaries of fc6.0 - alexnetgn_1x
!wget -nc https://www.dropbox.com/s/lk92j9eycivuryb/EarlyV_alexnetgn_1x_fc6.0_feature_idxs.pickle?dl=0
!wget -nc https://www.dropbox.com/s/8ukby7ijywpfaeh/pOTC_alexnetgn_1x_fc6.0_feature_idxs.pickle?dl=0
!wget -nc https://www.dropbox.com/s/cd67e67df3l8ge9/aOTC_alexnetgn_1x_fc6.0_feature_idxs.pickle?dl=0

#random features
!wget -nc https://www.dropbox.com/s/w1awqak2ud0tqvc/classifier1_random_feature_idxs.pickle?dl=0

#pretrained alexnet - all layers
!wget -nc https://www.dropbox.com/s/830zablobivnp21/pretrained_alexnet_feature_idxs.zip?dl=0
!unzip pretrained_alexnet_feature_idxs.zip?dl=0

#random layers of pretrained alexnet
!wget -nc https://www.dropbox.com/s/nevwqchekmzxsvg/random_feature_idxs.zip?dl=0
!unzip random_feature_idxs.zip?dl=0

In [ ]:
import pickle

def dict_save(dict, file_name):
    with open(file_name + '.pickle', 'wb') as f:
        pickle.dump(dict, f, pickle.HIGHEST_PROTOCOL)

def dict_load(dict_name):
    with open(dict_name , 'rb') as f:
        return pickle.load(f)

In [ ]:
#loading dicts
'''
EarlyV_pretrained_alexnet_classifier1_feature_idxs = dict_load('EarlyV_pretrained_alexnet_classifier1_feature_idxs.pickle?dl=0')
pOTC_pretrained_alexnet_classifier1_feature_idxs = dict_load('pOTC_pretrained_alexnet_classifier1_feature_idxs.pickle?dl=0')
aOTC_pretrained_alexnet_classifier1_feature_idxs = dict_load('aOTC_pretrained_alexnet_classifier1_feature_idxs.pickle?dl=0')

EarlyV_alexnetgn_1x_fc6_0_feature_idxs = dict_load('EarlyV_alexnetgn_1x_fc6.0_feature_idxs.pickle?dl=0')
pOTC_alexnetgn_1x_fc6_0_feature_idxs = dict_load('pOTC_alexnetgn_1x_fc6.0_feature_idxs.pickle?dl=0')
aOTC_alexnetgn_1x_fc6_0_feature_idxs = dict_load('aOTC_alexnetgn_1x_fc6.0_feature_idxs.pickle?dl=0')

random_feature_idxs = dict_load('classifier1_random_feature_idxs.pickle?dl=0')

#keys are percentages - of highest features
#higher percentage, more features included
#random_feature_idxs.keys()
'''

In [ ]:
#check to see if feature idxs are different
'''
x = EarlyV_pretrained_alexnet_classifier1_feature_idxs[.05]
y = pOTC_pretrained_alexnet_classifier1_feature_idxs[.05]
z = aOTC_pretrained_alexnet_classifier1_feature_idxs[.05]

print(len(x))
print(len(y))
print(len(z))

common = []
for i in y:
  if i in z:
    common.append(i)

len(common)
'''

Feature Editor

In [ ]:
import torch
import torch.nn as nn 

class FeatureEditor(nn.Module):
    def __init__(self, model, layers, masks, return_features=False):
        super().__init__()
        self.model = model
        self.layers = [layers] if isinstance(layers, str) else layers
        self.masks = [masks] if not isinstance(layers, list) else masks
        self.return_features = return_features
        self._features = {layer: torch.empty(0) for layer in layers}
        self.hooks = {}

    def hook_layers(self):        
        self.remove_hooks()
        for layer_id,mask in zip(self.layers, self.masks):
            layer = dict([*self.model.named_modules()])[layer_id]
            self.hooks[layer_id] = layer.register_forward_hook(self.edit_outputs_hook(layer_id,mask))
    
    def remove_hooks(self):
        for layer_id in self.layers:
            self._features[layer_id] = torch.empty(0)
            if layer_id in self.hooks:
                self.hooks[layer_id].remove()
                del self.hooks[layer_id]
    
    def __enter__(self, *args): 
        self.hook_layers()
        return self
    
    def __exit__(self, *args): 
        self.remove_hooks()
            
    def edit_outputs_hook(self, layer_id, mask):
        def fn(_, __, output):
            modified_output = output * mask
            self._features[layer_id] = modified_output
            #changed this below
            modified_output.to(device)
            #changed this above
            return modified_output
        return fn

    def forward(self, x):
        out = self.model(x)
        if self.return_features:
            return self._features
        else:
            return out
        
def generate_mask(shape, units):
    mask = torch.ones(shape).flatten()
    mask[units] = 0
    return mask.reshape(shape).unsqueeze(0)


import torch
import torch.nn as nn
from torchvision import models
from IPython.core.debugger import set_trace

class FeatureExtractor(nn.Module):
    def __init__(self, model, layers, detach=True, clone=True, device='cpu'):
        super().__init__()
        self.model = model
        self.layers = [layers] if isinstance(layers, str) else layers
        self.detach = detach
        self.clone = clone
        self.device = device
        self._features = {layer: torch.empty(0) for layer in layers}        
        self.hooks = {}
        
    def hook_layers(self):        
        self.remove_hooks()
        for layer_id in self.layers:
            layer = dict([*self.model.named_modules()])[layer_id]
            self.hooks[layer_id] = layer.register_forward_hook(self.save_outputs_hook(layer_id))
    
    def remove_hooks(self):
        for layer_id in self.layers:
            self._features[layer_id] = torch.empty(0)
            if layer_id in self.hooks:
                self.hooks[layer_id].remove()
                del self.hooks[layer_id]
    
    def __enter__(self, *args): 
        self.hook_layers()
        return self
    
    def __exit__(self, *args): 
        self.remove_hooks()
        
    def save_outputs_hook(self, layer_id):
        def fn(_, __, output):
            if self.detach: output = output.detach()
            if self.clone: output = output.clone()
            if self.device: output = output.to(self.device)
            self._features[layer_id] = output
        return fn

    def forward(self, x):
        _ = self.model(x)
        return self._features
    
def get_layers(model, parent_name='', layer_info=[]):
    for module_name, module in model.named_children():
        layer_name = parent_name + '.' + module_name
        if len(list(module.named_children())):
            layer_info = get_layers(module, layer_name, layer_info=layer_info)
        else:
            layer_info.append(layer_name.strip('.'))
    
    return layer_info

def get_layer(m, layers):
    layer = layers.pop(0)
    m = getattr(m, layer)
    if len(layers) > 0:
        return get_layer(m, layers)
    return m

def get_layer_type(model, layer_name):
    m = get_layer(model, layer_name.split("."))
    return m.__class__.__name__
            
def convert_relu_layers(parent):
    for child_name, child in parent.named_children():
        if isinstance(child, nn.ReLU) and child.inplace==True:
            setattr(parent, child_name, nn.ReLU(inplace=False))
        elif len(list(child.children())) > 0:
            convert_relu_layers(child)

Validation

In [ ]:
#downloading validation dataser
!wget -c https://www.dropbox.com/s/6vu07wtshpqpcr2/val.tar.gz
!tar -xf val.tar.gz
!rm val.tar.gz

In [ ]:
import torch 
from enum import Enum
from fastprogress.fastprogress import progress_bar 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

@torch.no_grad()
def validate(model, val_loader):
  num_images = len(val_loader.dataset)
  assert num_images==50000, f"Expected 50K images, got {num_images}"  
  criterion = torch.nn.CrossEntropyLoss()
  model.eval()
  losses = AverageMeter('Loss', ':.4e', Summary.NONE)
  top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
  top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)

  for i, (images, target) in enumerate(progress_bar(val_loader)):
      images = images.cuda(device, non_blocking=True)
      target = target.cuda(device, non_blocking=True)

      # compute output
      output = model(images)
      loss = criterion(output, target)

      # measure accuracy and record loss
      acc1, acc5 = accuracy(output, target, topk=(1, 5))
      losses.update(loss.item(), images.size(0))
      top1.update(acc1[0], images.size(0))
      top5.update(acc5[0], images.size(0))

  return top1.avg.item(), top5.avg.item(), losses.avg

class Summary(Enum):
    NONE = 0
    AVERAGE = 1
    SUM = 2
    COUNT = 3

class AverageMeter(object):
    def __init__(self, name, fmt=':f', summary_type=Summary.AVERAGE):
        self.name = name
        self.fmt = fmt
        self.summary_type = summary_type
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def all_reduce(self):
        total = torch.FloatTensor([self.sum, self.count])
        dist.all_reduce(total, dist.ReduceOp.SUM, async_op=False)
        self.sum, self.count = total.tolist()
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
    def summary(self):
        fmtstr = ''
        if self.summary_type is Summary.NONE:
            fmtstr = ''
        elif self.summary_type is Summary.AVERAGE:
            fmtstr = '{name} {avg:.3f}'
        elif self.summary_type is Summary.SUM:
            fmtstr = '{name} {sum:.3f}'
        elif self.summary_type is Summary.COUNT:
            fmtstr = '{name} {count:.3f}'
        else:
            raise ValueError('invalid summary type %r' % self.summary_type)
        
        return fmtstr.format(**self.__dict__)

def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
from torchvision import models, datasets, transforms 

transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])


image_folder = './val'
dataset = datasets.ImageFolder(image_folder, transform=transform)
val_loader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=256, 
                                         shuffle=False, 
                                         num_workers=2, 
                                         pin_memory=True)
dataset

Dataset ImageFolder
    Number of datapoints: 50000
    Root location: ./val
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

Defining models used

In [ ]:
model_1 = models.alexnet(pretrained=True)
model_1.to(device)


Running Validation

In [ ]:
import numpy as np
from google.colab import files #for downloading files to downloads
#percentages to be used - ie: lesion this percentage of features
percentages = np.array([.05, .10, .15, .30, .50, .70, .90])
layer_names = get_layers(model_1)
layer_names

All Layers

In [ ]:
#getting output map shapes
output_map_shapes = {}
for layer_name in layer_names:
  layer_name_string = layer_name.replace(".", "")
  feature_idxs = dict_load("pretrained_alexnet_feature_idxs/EarlyV_pretrained_alexnet_"+layer_name_string+"_feature_idxs.pickle")
  with FeatureExtractor(model_1, [layer_name]) as extractor:
    extractor.to(device)
    dummy_images = torch.rand(10,3,224,224)
    dummy_images = dummy_images.to(device)
    output = extractor(dummy_images)
    with torch.no_grad():
      features = output[layer_name]
  output_map_shape = features.shape[1:]
  output_map_shapes[layer_name] = output_map_shape    

In [ ]:
#getting mask units
brain_regions = ['EarlyV', 'pOTC', 'aOTC']
mask_units_dict = {}
for region in brain_regions:
  print("region:", region)
  for layer_name in layer_names:
    for percentage in percentages:
      print(percentage)
      layer_name_string = layer_name.replace(".", "")
      mask_units_dict[region+"_"+layer_name+"_"+str(percentage)] = dict_load("pretrained_alexnet_feature_idxs/"+region+"_pretrained_alexnet_"+layer_name_string+"_feature_idxs.pickle")[percentage]
mask_units_dict.keys()

In [ ]:
#getting mask units for random
region = 'random'
for layer_name in layer_names:
  for percentage in percentages:
    print(percentage)
    layer_name_string = layer_name.replace(".", "")
    mask_units_dict[region+"_"+layer_name+"_"+str(percentage)] = dict_load("random_feature_idxs/"+layer_name_string+"_random_feature_idxs.pickle")[percentage]
mask_units_dict.keys()

In [ ]:
#generating and saving masks
masks = {}
brain_regions = ['EarlyV', 'pOTC', 'aOTC', 'random']
for region in brain_regions:
  for layer_name in layer_names:
    for percentage in percentages:
      mask_units = mask_units_dict[region+"_"+layer_name+"_"+str(percentage)]
      output_map_shape = output_map_shapes[layer_name]
      mask = generate_mask(output_map_shape, mask_units)
      masks[region+"_"+layer_name+"_"+str(percentage)] = mask
masks.keys()

# test multi-layer lesioning

In [ ]:
layer_names[0:1], percentages

(['features.0'], array([0.05, 0.1 , 0.15, 0.3 , 0.5 , 0.7 , 0.9 ]))

In [ ]:
# first a single layer
brain_region = 'EarlyV'
percentage = 0.05
test_masks = [
  masks[brain_region+"_"+layer_names[0]+"_"+str(percentage)].to(device)
]
test_layers = layer_names[0:1] # <- indexing with range 0:1 ensures we get a "list" as ouput, in this case a list of only one layer_name

with FeatureEditor(model_1, test_layers, test_masks, return_features=False) as editor:
  dummy_images = torch.rand(10,3,224,224).to(device)
  output = editor(dummy_images)
output.shape

torch.Size([10, 1000])

In [ ]:
# now multiple layers
brain_region = 'EarlyV'
percentage = 0.05
test_masks = [
  masks[brain_region+"_"+layer_names[0]+"_"+str(percentage)].to(device),
  masks[brain_region+"_"+layer_names[1]+"_"+str(percentage)].to(device)
]
test_layers = layer_names[0:2] # python excludes the last item in a range; so this gives us layers 0,1

with FeatureEditor(model_1, test_layers, test_masks, return_features=False) as editor:
  dummy_images = torch.rand(10,3,224,224).to(device)
  output = editor(dummy_images)
output.shape

torch.Size([10, 1000])

# Use list comprehension to get list of masks



In [ ]:
layer_masks = [masks[f"{brain_region}_{layer_name}_{percentage}"].to(device) for layer_name in layer_names]
len(layer_masks)

21

#All Layer Lesioning

In [ ]:
#EarlyV all layers lesioning
brain_region = 'EarlyV'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

for i in range(percentages.size):
  percentage = percentages[i]

  layer_masks = [masks[f"{brain_region}_{layer_name}_{percentage}"].to(device) for layer_name in layer_names]

  with FeatureEditor(model_1, layer_names, layer_masks, return_features=False) as editor:
    top1, top5, loss = validate(editor, val_loader)

  #appends accuracies and losses to list
  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss
  
EarlyV_all_layers = {}
EarlyV_all_layers['percentages'] = percentages
EarlyV_all_layers['top1s'] = top1s
EarlyV_all_layers['top5s'] = top5s
EarlyV_all_layers['losses'] = losses

dict_save(EarlyV_all_layers, brain_region+"_all_layers_accuracies")
#downloads to downloads
files.download(brain_region+"_all_layers_accuracies.pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#pOTC all layers lesioning
brain_region = 'pOTC'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

for i in range(percentages.size):
  percentage = percentages[i]

  layer_masks = [masks[f"{brain_region}_{layer_name}_{percentage}"].to(device) for layer_name in layer_names]

  with FeatureEditor(model_1, layer_names, layer_masks, return_features=False) as editor:
    top1, top5, loss = validate(editor, val_loader)

  #appends accuracies and losses to list
  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss
  
pOTC_all_layers = {}
pOTC_all_layers['percentages'] = percentages
pOTC_all_layers['top1s'] = top1s
pOTC_all_layers['top5s'] = top5s
pOTC_all_layers['losses'] = losses

dict_save(pOTC_all_layers, brain_region+"_all_layers_accuracies")
#downloads to downloads
files.download(brain_region+"_all_layers_accuracies.pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#aOTC all layers lesioning
brain_region = 'aOTC'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

for i in range(percentages.size):
  percentage = percentages[i]

  layer_masks = [masks[f"{brain_region}_{layer_name}_{percentage}"].to(device) for layer_name in layer_names]

  with FeatureEditor(model_1, layer_names, layer_masks, return_features=False) as editor:
    top1, top5, loss = validate(editor, val_loader)

  #appends accuracies and losses to list
  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss
  
aOTC_all_layers = {}
aOTC_all_layers['percentages'] = percentages
aOTC_all_layers['top1s'] = top1s
aOTC_all_layers['top5s'] = top5s
aOTC_all_layers['losses'] = losses

dict_save(aOTC_all_layers, brain_region+"_all_layers_accuracies")
#downloads to downloads
files.download(brain_region+"_all_layers_accuracies.pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#random all layers lesioning
brain_region = 'random'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

for i in range(percentages.size):
  percentage = percentages[i]
  layer_masks = [masks[f"{brain_region}_{layer_name}_{percentage}"].to(device) for layer_name in layer_names]

  with FeatureEditor(model_1, layer_names, layer_masks, return_features=False) as editor:
    top1, top5, loss = validate(editor, val_loader)

  #appends accuracies and losses to list
  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss
  
random_all_layers = {}
random_all_layers['percentages'] = percentages
random_all_layers['top1s'] = top1s
random_all_layers['top5s'] = top5s
random_all_layers['losses'] = losses

dict_save(random_all_layers, brain_region+"_all_layers_accuracies")
#downloads to downloads
files.download(brain_region+"_all_layers_accuracies.pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Multi-layer-lesion function

In [ ]:
def run_multi_layer_lesioning(model, val_loader, brain_region, masks, layer_names):
  #what the scores are going to be stored in
  top1s = np.zeros(percentages.size)
  top5s = np.zeros(percentages.size)
  losses = np.zeros(percentages.size)

  for i in range(percentages.size):
    percentage = percentages[i]
    layer_masks = [masks[f"{brain_region}_{layer_name}_{percentage}"].to(device) for layer_name in layer_names]

    with FeatureEditor(model, layer_names, layer_masks, return_features=False) as editor:
      top1, top5, loss = validate(editor, val_loader)

    #appends accuracies and losses to list
    top1s[i] = top1
    top5s[i] = top5
    losses[i] = loss
    
  results = {}
  results['brain_region'] = brain_region
  results['layer_names'] = layer_names
  results['percentages'] = percentages
  results['top1s'] = top1s
  results['top5s'] = top5s
  results['losses'] = losses

  return results

In [ ]:
'''
#model_1 - pretrained alexnet
layer_name = 'classifier.1'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

#get output map shape
with FeatureExtractor(model_1, [layer_name]) as extractor:
  extractor.to(device)
  dummy_images = torch.rand(10,3,224,224)
  dummy_images = dummy_images.to(device)
  output = extractor(dummy_images)
  with torch.no_grad():
    features = output[layer_name]
output_map_shape = features.shape[1:]


brain_regions_dict = {}
brain_regions_dict['EarlyV'] = EarlyV_pretrained_alexnet_classifier1_feature_idxs
brain_regions_dict['pOTC'] = pOTC_pretrained_alexnet_classifier1_feature_idxs
brain_regions_dict['aOTC'] = aOTC_pretrained_alexnet_classifier1_feature_idxs

for region in brain_regions_dict.keys():
  mask_units_dict = brain_regions_dict[region]

  for i in range(percentages.size):
    #make mask for features
    percentage_features_lesioned = percentages[i]
    mask_units = mask_units_dict[round(percentage_features_lesioned,2)]
    mask = generate_mask(output_map_shape, mask_units)
    mask = mask.to(device)

    with FeatureEditor(model_1, [layer_name], [mask], return_features=False) as editor:
      top1, top5, loss = validate(editor, val_loader)
  
    #appends accuracies and losses to list
    top1s[i] = top1
    top5s[i] = top5
    losses[i] = loss

  #making and saving dicts
  pretrained_alexnet_classifier1_accuracies = {}

  pretrained_alexnet_classifier1_accuracies["top1s"] = top1s
  pretrained_alexnet_classifier1_accuracies["top5s"] = top5s
  pretrained_alexnet_classifier1_accuracies["losses"] = losses

  dict_save(pretrained_alexnet_classifier1_accuracies, region+"_pretrained_alexnet_classifier1_accuracies")
  #downloads to downloads
  files.download(region+"_pretrained_alexnet_classifier1_accuracies.pickle")
  '''

In [ ]:
'''
#model_2 - alexnetgn - aOTC
layer_name = 'fc6.0'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

region = 'aOTC'
#get output map shape
with FeatureExtractor(model_2, [layer_name]) as extractor:
  extractor.to(device)
  dummy_images = torch.rand(10,3,224,224)
  dummy_images = dummy_images.to(device)
  output = extractor(dummy_images)
  with torch.no_grad():
    features = output[layer_name]
output_map_shape = features.shape[1:]

for i in range(percentages.size):
  #make mask for features
  percentage_features_lesioned = percentages[i]
  mask_units = aOTC_alexnetgn_1x_fc6_0_feature_idxs[round(percentage_features_lesioned,2)]
  mask = generate_mask(output_map_shape, mask_units)
  mask = mask.to(device)
  
  with FeatureEditor(model_2, [layer_name], [mask], return_features=False) as editor:
      top1, top5, loss = validate(editor, val_loader)

  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss

#making and saving dicts
alexnetgn_1x_fc6_0_accuracies = {}

alexnetgn_1x_fc6_0_accuracies["top1s"] = top1s
alexnetgn_1x_fc6_0_accuracies["top5s"] = top5s
alexnetgn_1x_fc6_0_accuracies["losses"] = losses

dict_save(alexnetgn_1x_fc6_0_accuracies, region+"_alexnetgn_1x_fc6_0_accuracies")
#downloads to downloads
files.download(region+"_alexnetgn_1x_fc6_0_accuracies.pickle")
'''

In [ ]:
'''
#model_1 - pretrained alexnet - random features lesioned
layer_name = 'classifier.1'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

#get output map shape
with FeatureExtractor(model_1, [layer_name]) as extractor:
  extractor.to(device)
  dummy_images = torch.rand(10,3,224,224)
  dummy_images = dummy_images.to(device)
  output = extractor(dummy_images)
  with torch.no_grad():
    features = output[layer_name]
output_map_shape = features.shape[1:]

mask_units_dict = random_feature_idxs

for i in range(percentages.size):
  #make mask for features
  percentage_features_lesioned = percentages[i]
  mask_units = mask_units_dict[round(percentage_features_lesioned,2)]
  mask = generate_mask(output_map_shape, mask_units)
  mask = mask.to(device)

  with FeatureEditor(model_1, [layer_name], [mask], return_features=False) as editor:
      top1, top5, loss = validate(editor, val_loader)

  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss

random_pretrained_alexnet_classifier1_accuracies = {}
random_pretrained_alexnet_classifier1_accuracies["top1s"] = top1s
random_pretrained_alexnet_classifier1_accuracies["top5s"] = top5s
random_pretrained_alexnet_classifier1_accuracies["losses"] = losses

dict_save(random_pretrained_alexnet_classifier1_accuracies, region+"_random_pretrained_alexnet_classifier1_accuracies")
#downloads to downloads
files.download(region+"_random_pretrained_alexnet_classifier1_accuracies.pickle")
'''

In [ ]:
'''
#model_2 - alexnetgn 1x
layer_name = 'fc6.0'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

#get output map shape
with FeatureExtractor(model_2, [layer_name]) as extractor:
  extractor.to(device)
  dummy_images = torch.rand(10,3,224,224)
  dummy_images = dummy_images.to(device)
  output = extractor(dummy_images)
  with torch.no_grad():
    features = output[layer_name]
output_map_shape = features.shape[1:]


brain_regions_dict = {}
brain_regions_dict['EarlyV'] = EarlyV_alexnetgn_1x_fc6_0_feature_idxs
brain_regions_dict['pOTC'] = pOTC_alexnetgn_1x_fc6_0_feature_idxs
brain_regions_dict['aOTC'] = aOTC_alexnetgn_1x_fc6_0_feature_idxs

for region in brain_regions_dict.keys():
  mask_units_dict = brain_regions_dict[region]

  for i in range(percentages.size):
    #make mask for features
    percentage_features_lesioned = percentages[i]
    mask_units = mask_units_dict[round(percentage_features_lesioned,2)]
    mask = generate_mask(output_map_shape, mask_units)
    mask = mask.to(device)

    with FeatureEditor(model_2, [layer_name], [mask], return_features=False) as editor:
      top1, top5, loss = validate(editor, val_loader)
  
    #appends accuracies and losses to list
    top1s[i] = top1
    top5s[i] = top5
    losses[i] = loss

  #making and saving dicts
  alexnetgn_1x_fc6_0_accuracies = {}

  alexnetgn_1x_fc6_0_accuracies["top1s"] = top1s
  alexnetgn_1x_fc6_0_accuracies["top5s"] = top5s
  alexnetgn_1x_fc6_0_accuracies["losses"] = losses

  dict_save(alexnetgn_1x_fc6_0_accuracies, region+"_alexnetgn_1x_fc6_0_accuracies")
  #downloads to downloads
  files.download(region+"_alexnetgn_1x_fc6_0_accuracies.pickle")
'''

In [ ]:
'''
#model_2 - alexnetgn 1x - random
layer_name = 'fc6.0'

#what the scores are going to be stored in
top1s = np.zeros(percentages.size)
top5s = np.zeros(percentages.size)
losses = np.zeros(percentages.size)

#get output map shape
with FeatureExtractor(model_2, [layer_name]) as extractor:
  extractor.to(device)
  dummy_images = torch.rand(10,3,224,224)
  dummy_images = dummy_images.to(device)
  output = extractor(dummy_images)
  with torch.no_grad():
    features = output[layer_name]
output_map_shape = features.shape[1:]

mask_units_dict = random_feature_idxs

for i in range(percentages.size):
  #make mask for features
  percentage_features_lesioned = percentages[i]
  mask_units = mask_units_dict[round(percentage_features_lesioned,2)]
  mask = generate_mask(output_map_shape, mask_units)
  mask = mask.to(device)

  with FeatureEditor(model_2, [layer_name], [mask], return_features=False) as editor:
      top1, top5, loss = validate(editor, val_loader)

  top1s[i] = top1
  top5s[i] = top5
  losses[i] = loss
alexnetgn_1x_fc6_0_accuracies
random_alexnetgn_1x_fc6_0_accuracies = {}
random_alexnetgn_1x_fc6_0_accuracies["top1s"] = top1s
random_alexnetgn_1x_fc6_0_accuracies["top5s"] = top5s
random_alexnetgn_1x_fc6_0_accuracies["losses"] = losses

dict_save(random_alexnetgn_1x_fc6_0_accuracies, region+"_random_alexnetgn_1x_fc6_0_accuracies")
#downloads to downloads
files.download(region+"_random_alexnetgn_1x_fc6_0_accuracies.pickle")
'''